In [33]:
import sys
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
from gezi.common import *
from src.config import *
from src.preprocess import *
from src.eval import *
gezi.init_flags()
gezi.set_pandas()

In [34]:
V = 7
root = f'../working/offline/{V}/0'
# pairwise two tower, recall model
# pt_model = 'all-mpnet-base-v2.flag-pairwise14-2-pre_ext_emlm_mlm.ep1'
# p2t_model = 'all-mpnet-base-v2.flag-pairwise14-2'
pt_model = 'all-mpnet-base-v2.flag-pairwise14-2-pre_mlm3'
# pt_model = 'all-mpnet-base-v2.flag-pairwise14-2'
# pt_model2 = 'pmminilm.flag-pairwise14-2-pre_emlm_mlm-mmnilm'
# pt_model2 = 'pmminilm.flag-pairwise14-2-pre_mlm3-pmminilm'
pt_model2 = 'pmminilm.flag-pairwise14-2'
# pairwise concat, rank model

# context model
# c_model = 'deberta-v3-small.flag-context4-2-d'
# c_model2 = 'lsg-mminilm.flag-context4-3-d-s-mminilm'
# c_model = 'deberta-v3-small.flag-context4-2-d'
# c_model = 'deberta-v3-small.flag-context4-2-d-oof-ft'
c_model = 'deberta-v3-small.flag-context4-2-d-oof-ft.list_train_ordered'
# c_model = 'deberta-v3-small.flag-context4-2-d-extpred-ft.0804'

pc_model = 'deberta-v3-small.flag-pairwise14-4-cat-insert-extpred-ft.neg_rand_prob-0.neg_strategy-rand-sample'

In [35]:
df_train = pd.read_feather('../working/train.fea')
df_train = df_train[df_train.fold==0]

In [36]:
xp = gezi.load(f'{root}/{pt_model}.eval/valid.pkl')

In [37]:
xp.keys()

dict_keys(['id', 'cell_id', 'cid', 'pred', 'max_prob', 'max_sim', 'cls_pred', 'n_words', 'match_rank', 'match_code', 'n_code_cell', 'n_cell', 'probs', 'sims'])

In [38]:
xp2 = gezi.load(f'{root}/{pt_model2}.eval/valid.pkl')

In [39]:
xc = gezi.load(f'{root}/{c_model}.eval/valid.pkl')

In [40]:
# calc_metric(xc, 'reg_pred')

In [41]:
xc.keys()

dict_keys(['pred', 'cls_pred', 'cid', 'pos', 'pred_id', 'id', 'cell_id', 'cell_type', 'rank', 'rel_rank', 'n_code_cell', 'n_markdown_cell', 'n_cell', 'n_words', 'reg_pred', 'cls_pred_ori', 'cls2_pred'])

In [42]:
d = pd.DataFrame(gezi.batch2list(xc))
dg = d.groupby('pred_id')
max_preds = dg['pred'].max()
min_preds = dg['pred'].min()
max_cls_preds = dg['cls_pred'].max()
min_cls_preds = dg['cls_pred'].min()

In [43]:
gezi.sort_dict_byid_(xp, 'cid')
gezi.sort_dict_byid_(xp2, 'cid')
gezi.sort_dict_byid_(xc, 'cid')

In [44]:
LABEL_COL = 'rel_rank'
def diff_feats(x, l,r, name):
  if isinstance(l, str):
    x[f'{name}_diff'] = x[l] - x[r]
  else:
    x[f'{name}_diff'] = l - r
  x[f'abs_{name}_diff'] = abs(x[f'{name}_diff'])

def gen_pfeat(x_):
  topn = 10
  x = {}
  keys = ['pred', 'cls_pred']
  x = {k: x_[k] for k in keys}
  x['rank_pred'] = x['pred'] * (1 + x_['n_code_cell']) - 0.5
  x['cls_rank_pred'] = x['cls_pred'] * (1 + x_['n_code_cell']) - 0.5
  x['min_prob'] = x_['probs'].min()
  x['min_sim'] = x_['sims'].min()
  x['var_prob'] = np.var(x_['probs'])
  x['var_sim'] = np.var(x_['sims'])
  idxes = (-x_['probs']).argsort()
  for i in range(topn):
    if i < len(idxes):
      if i > 0:
        x[f'top_pred_{i}'] = (idxes[i] + 0.5) / (x_['n_code_cell'] + 1)
      x[f'top_prob_{i}'] = x_['probs'][idxes[i]]
      x[f'top_sim_{i}'] = x_['sims'][idxes[i]]
    else:
      x[f'top_pred_{i}'] = -1
      x[f'top_prob_{i}'] = -1
      x[f'top_sim_{i}'] = -1
  
  diff_feats(x, 'cls_pred', 'pred', 'cls')
  return x

def gen_cfeat(x_):
  topn = 4
  x = {}
  keys = [
    'pred', 
    'cls_pred', 
    'reg_pred', 
    'cls2_pred', 
    # 'pos'
    ]
  x = {k: x_[k] for k in keys}
  x['rank_pred'] = x['pred'] * FLAGS.num_classes - 0.5
  x['cls_rank_pred'] = x['cls_pred'] * FLAGS.num_classes - 0.5
  x['cls2_rank_pred'] = x['cls2_pred'] * FLAGS.num_classes - 0.5
  x['reg_rank_pred'] = x['cls_pred'] * FLAGS.num_classes - 0.5
  preds = x_['cls_pred_ori']
  probs = gezi.softmax(preds)
  x['min_prob'] = probs.min()
  x['var_prob'] = np.var(probs)
  idxes = (-probs).argsort()

  for i in range(topn):
    if i < len(idxes):
      if i > 0:
        x[f'top_pred_{i}'] = (idxes[i] + 0.5) / FLAGS.num_classes
      x[f'top_prob_{i}'] = probs[idxes[i]]
    else:
      x[f'top_pred_{i}'] = -1
      x[f'top_prob_{i}'] = -1
  diff_feats(x, 'cls_pred', 'reg_pred', 'cls_reg')
  diff_feats(x, 'cls_pred', 'cls2_pred', 'cls_cls2')
  diff_feats(x, 'cls2_pred', 'reg_pred', 'cls2_reg')
  
  # group_id = x_['pred_id']
  # x['max_pred'] = max_preds[group_id]
  # x['min_pred'] = min_preds[group_id]
  # x['pred_ratio'] = (x['pred'] - x['min_pred']) / (x['max_pred'] - x['min_pred']) if (x['max_pred'] - x['min_pred']) else 1
  # x['max_cls_pred'] = max_cls_preds[group_id]
  # x['min_cls_pred'] = min_cls_preds[group_id]
  # x['cls_pred_ratio'] = (x['cls_pred'] - x['min_cls_pred']) / (x['max_cls_pred'] - x['min_cls_pred']) if (x['max_cls_pred'] - x['min_cls_pred']) else 1
  
  return x

def gen_feats():
  xs = gezi.batch2list(xp)
  p_feats = [gen_pfeat(x) for x in tqdm(xs, desc=f'gen_pfeats:xp')]
  xs = gezi.batch2list(xp2)
  p2_feats = [gen_pfeat(x) for x in tqdm(xs, desc=f'gen_pfeats:xp2')]
  xs = gezi.batch2list(xc)
  c_feats = [gen_cfeat(x) for x in tqdm(xs, desc=f'gen_cfeats:xc')]
  feats = []
  for i in tqdm(range(len(xs)), desc='merge_feats'):
    fe1 = p_feats[i]
    fe2= p2_feats[i]
    fec = c_feats[i]
    
    fe = {}
    fe['code_ratio'] = xp['n_code_cell'][i] / xp['n_cell'][i]
    diff_feats(fe, fe1['pred'], fe2['pred'], 'ps')
    diff_feats(fe, fe1['cls_pred'], fe2['cls_pred'], 'ps_cls')
    diff_feats(fe, fe1['pred'], fec['reg_pred'], 'pc_reg')
    diff_feats(fe, fe2['pred'], fec['reg_pred'], 'p2c_reg')
    diff_feats(fe, fe1['pred'], fec['cls_pred'], 'pc_cls')
    diff_feats(fe, fe2['pred'], fec['cls_pred'], 'p2c_cls')
    diff_feats(fe, fe1['pred'], fec['cls2_pred'], 'pc_cls2')
    diff_feats(fe, fe2['pred'], fec['cls2_pred'], 'p2c_cls2')
    diff_feats(fe, fe1['cls_pred'], fec['reg_pred'], 'pc_cls_reg')
    diff_feats(fe, fe2['cls_pred'], fec['reg_pred'], 'p2c_cls_reg')
    diff_feats(fe, fe1['cls_pred'], fec['cls_pred'], 'pc_cls_cls')
    diff_feats(fe, fe2['cls_pred'], fec['cls_pred'], 'p2c_cls_cls')

    fe1 = gezi.dict_prefix(fe1, 'p_')
    fe.update(fe1)
    fe2 = gezi.dict_prefix(fe2, 'p2_')
    fe.update(fe2)
    fec = gezi.dict_prefix(fec, 'c_')
    fe.update(fec)
    
    keys = [
     'id', 'cell_id', 'cid',
     'n_words', 'n_code_cell', 'n_cell'
    ]
    for key in keys:
      fe[key] = xp[key][i]
      
    #  ic(feat)
    feats.append(fe)
  #  break
  dfeats = pd.DataFrame(feats)
  return dfeats

In [45]:
dfeats = gen_feats()

gen_pfeats:xp:   0%|          | 0/424943 [00:00<?, ?it/s]

gen_pfeats:xp2:   0%|          | 0/424943 [00:00<?, ?it/s]

gen_cfeats:xc:   0%|          | 0/424943 [00:00<?, ?it/s]

merge_feats:   0%|          | 0/424943 [00:00<?, ?it/s]

In [46]:
dfeats

,code_ratio,ps_diff,abs_ps_diff,ps_cls_diff,abs_ps_cls_diff,pc_reg_diff,abs_pc_reg_diff,p2c_reg_diff,abs_p2c_reg_diff,pc_cls_diff,abs_pc_cls_diff,p2c_cls_diff,abs_p2c_cls_diff,pc_cls2_diff,abs_pc_cls2_diff,p2c_cls2_diff,abs_p2c_cls2_diff,pc_cls_reg_diff,abs_pc_cls_reg_diff,p2c_cls_reg_diff,abs_p2c_cls_reg_diff,pc_cls_cls_diff,abs_pc_cls_cls_diff,p2c_cls_cls_diff,abs_p2c_cls_cls_diff,p_pred,p_cls_pred,p_rank_pred,p_cls_rank_pred,p_min_prob,p_min_sim,p_var_prob,p_var_sim,p_top_prob_0,p_top_sim_0,p_top_pred_1,p_top_prob_1,p_top_sim_1,p_top_pred_2,p_top_prob_2,p_top_sim_2,p_top_pred_3,p_top_prob_3,p_top_sim_3,p_top_pred_4,p_top_prob_4,p_top_sim_4,p_top_pred_5,p_top_prob_5,p_top_sim_5,p_top_pred_6,p_top_prob_6,p_top_sim_6,p_top_pred_7,p_top_prob_7,p_top_sim_7,p_top_pred_8,p_top_prob_8,p_top_sim_8,p_top_pred_9,p_top_prob_9,p_top_sim_9,p_cls_diff,p_abs_cls_diff,p2_pred,p2_cls_pred,p2_rank_pred,p2_cls_rank_pred,p2_min_prob,p2_min_sim,p2_var_prob,p2_var_sim,p2_top_prob_0,p2_top_sim_0,p2_top_pred_1,p2_top_prob_1,p2_top_sim_1,p2_top_pred_2,p2_top_prob_2,p2_top_sim_2,p2_top_pred_3,p2_top_prob_3,p2_top_sim_3,p2_top_pred_4,p2_top_prob_4,p2_top_sim_4,p2_top_pred_5,p2_top_prob_5,p2_top_sim_5,p2_top_pred_6,p2_top_prob_6,p2_top_sim_6,p2_top_pred_7,p2_top_prob_7,p2_top_sim_7,p2_top_pred_8,p2_top_prob_8,p2_top_sim_8,p2_top_pred_9,p2_top_prob_9,p2_top_sim_9,p2_cls_diff,p2_abs_cls_diff,c_pred,c_cls_pred,c_reg_pred,c_cls2_pred,c_rank_pred,c_cls_rank_pred,c_cls2_rank_pred,c_reg_rank_pred,c_min_prob,c_var_prob,c_top_prob_0,c_top_pred_1,c_top_prob_1,c_top_pred_2,c_top_prob_2,c_top_pred_3,c_top_prob_3,c_cls_reg_diff,c_abs_cls_reg_diff,c_cls_cls2_diff,c_abs_cls_cls2_diff,c_cls2_reg_diff,c_abs_cls2_reg_diff,id,cell_id,cid,n_words,n_code_cell,n_cell
0,0.8889,0.0000,0.0000,-0.0237,0.0237,0.0038,0.0038,0.0038,0.0038,-0.0023,0.0023,-0.0023,0.0023,0.0006,0.0006,0.0006,0.0006,0.0127,0.0127,0.0364,0.0364,0.0067,0.0067,0.0304,0.0304,0.0556,0.0645,0.0000,0.0808,0.0000,0.5031,0.0867,0.0062,0.9432,0.7505,0.1667,0.0456,0.6840,0.3889,0.0096,0.6498,0.6111,0.0008,0.5963,0.2778,0.0006,0.5899,0.5000,0.0001,0.5559,0.9444,0.0000,0.5262,0.7222,0.0000,0.5204,0.8333,0.0000,0.5031,-1.0000,-1.0000,-1.0000,0.0090,0.0090,0.0556,0.0882,0.0000,0.2937,0.0000,0.2628,0.0591,0.0098,0.7826,0.5662,0.1667,0.1731,0.5248,0.3889,0.0202,0.4660,0.2778,0.0201,0.4658,0.6111,0.0025,0.4087,0.5000,0.0011,0.3866,0.9444,0.0003,0.3546,0.7222,0.0000,0.2721,0.8333,0.0000,0.2628,-1.0000,-1.0000,-1.0000,0.0326,0.0326,0.0548,0.0578,0.0518,0.0550,4.9806,5.2824,5.0000,5.2824,0.0000,0.0096,0.9844,0.1650,0.0203,0.0350,0.0009,0.9450,0.0002,0.0060,0.0060,0.0028,0.0028,0.0032,0.0032,0002115f48f982,9ec225f0,0002115f48f982\t9ec225f0,2,8,9
1,0.8333,0.0000,0.0000,0.0088,0.0088,-0.0035,0.0035,-0.0035,0.0035,0.0011,0.0011,0.0011,0.0011,-0.0005,0.0005,-0.0005,0.0005,-0.0124,0.0124,-0.0212,0.0212,-0.0078,0.0078,-0.0166,0.0166,0.9545,0.9456,10.0000,9.9017,0.0000,0.5679,0.0695,0.0052,0.9215,0.8047,0.8636,0.0752,0.7497,0.2273,0.0012,0.6594,0.0455,0.0010,0.6548,0.7727,0.0006,0.6450,0.6818,0.0002,0.6200,0.4091,0.0001,0.5992,0.1364,0.0001,0.5896,0.5909,0.0000,0.5820,0.5000,0.0000,0.5739,-0.0089,0.0089,0.9545,0.9368,10.0000,9.8051,0.0001,0.2140,0.0672,0.0044,0.9087,0.4671,0.8636,0.0623,0.3937,0.6818,0.0144,0.3535,0.7727,0.0043,0.3202,0.0455,0.0036,0.3155,0.2273,0.0026,0.3071,0.5909,0.0019,0.2982,0.1364,0.0011,0.2824,0.5000,0.0007,0.2700,0.3182,0.0004,0.2561,-0.0177,0.0177,0.9557,0.9534,0.9580,0.9550,95.0712,94.8416,95.0000,94.8416,0.0000,0.0098,0.9922,0.9650,0.0041,0.8650,0.0016,0.0450,0.0009,-0.0046,0.0046,-0.0016,0.0016,-0.0030,0.0030,00062ab8487156,96c8449c,00062ab8487156\t96c8449c,6,10,12
2,0.8333,0.4545,0.4545,0.2323,0.2323,-0.0811,0.0811,-0.5356,0.5356,-0.0824,0.0824,-0.5369,0.5369,-0.0950,0.0950,-0.5495,0.5495,-0.1452,0.1452,-0.3776,0.3776,-0.1466,0.1466,-0.3789,0.3789,0.5000,0.4358,5.0000,4.2939,0.0004,0.4950,0.0226,0.0026,0.5150,0.6508,0.2273,0.2409,0.6341,0.4091,0.0963,0.6140,0.5909,0.0622,0.6044,0.6818,0.0523,0.6006,0

In [47]:
df = dfeats.merge(df_train[['cid', 'ancestor_id', LABEL_COL]], on='cid')

In [48]:
gezi.set_fold(df, 5, 'ancestor_id')

In [49]:
reg_cols = [x for x in dfeats.columns if x not in ['id', 'cell_id', 'cid', LABEL_COL]]
cat_cols = []
cols = reg_cols + cat_cols
len(cols)

129

In [50]:
fold = 0
dvalid = df[df.fold==fold]
dtrain = df[df.fold!=fold]

In [51]:
X_train = dtrain[cols]
y_train = dtrain[LABEL_COL]
X_valid = dvalid[cols]
y_valid = dvalid[LABEL_COL]

In [52]:
from catboost import CatBoostRegressor, CatBoostRanker
cbt_params = {
              # 'bootstrap_type': 'Poisson',
              # 'learning_rate': 0.02,
              'learning_rate': 0.03,
              'reg_lambda': 7.960622217848342e-07, 
              'subsample': 0.7422597612762745,
              # 'bagging_temperature': 0.2,
              'max_depth': 10, 
              'early_stopping_rounds': 500,
              'n_estimators': 10000,
              'cat_features': [],
              'loss_function': 'MAE',
              'min_child_samples': 100,
              # 'task_type': 'GPU',
              # 'devices': '0',
              }

In [53]:
cbt_model = CatBoostRegressor(**cbt_params)
cbt_model.fit(X_train, y_train,
      eval_set=[(X_train, y_train), (X_valid, y_valid)],
              verbose=100,
              )  
dvalid['cb_pred'] = cbt_model.predict(dvalid[cols])
ic(calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cb_pred.values}))

0:	learn: 0.2522445	test: 0.2522445	test1: 0.2522014	best: 0.2522014 (0)	total: 80.7ms	remaining: 13m 26s
100:	learn: 0.0667309	test: 0.0667309	test1: 0.0684230	best: 0.0684230 (100)	total: 6.99s	remaining: 11m 24s
200:	learn: 0.0601964	test: 0.0601964	test1: 0.0622242	best: 0.0622242 (200)	total: 13.6s	remaining: 11m 3s
300:	learn: 0.0590556	test: 0.0590556	test1: 0.0614556	best: 0.0614556 (300)	total: 20.5s	remaining: 10m 59s
400:	learn: 0.0583226	test: 0.0583226	test1: 0.0610615	best: 0.0610615 (400)	total: 27.4s	remaining: 10m 56s
500:	learn: 0.0577011	test: 0.0577011	test1: 0.0607874	best: 0.0607874 (500)	total: 34.2s	remaining: 10m 49s
600:	learn: 0.0571735	test: 0.0571735	test1: 0.0606203	best: 0.0606203 (600)	total: 41.2s	remaining: 10m 44s
700:	learn: 0.0567438	test: 0.0567438	test1: 0.0605238	best: 0.0605238 (700)	total: 48.1s	remaining: 10m 37s
800:	learn: 0.0563123	test: 0.0563123	test1: 0.0604505	best: 0.0604505 (800)	total: 55.1s	remaining: 10m 32s
900:	learn: 0.0558635	t

[08/05/22 19:42:16] 147506464.py:7 in <module>
                    calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cb_pred.values}): 0.9155755035269483


0.9155755035269483

In [54]:
gezi.plot.feature_importance(cbt_model, topn=20)

In [55]:
f = gezi.plot.feature_importance_df(cbt_model)

In [56]:
f.head()

,feat_name,importanace
0,p_cls_pred,13.0447
1,c_reg_pred,7.2054
2,c_pred,4.8333
3,c_cls_rank_pred,4.5248
4,p_pred,4.1334


In [57]:
f[f.feat_name=='c_cls_pred']

,feat_name,importanace
5,c_cls_pred,3.7643


In [58]:
f[f.feat_name=='c_reg_pred']

,feat_name,importanace
1,c_reg_pred,7.2054


In [59]:
f[f.feat_name=='c_pos']

,feat_name,importanace


In [60]:
f[f.feat_name=='c_max_pred']

,feat_name,importanace


In [61]:
import lightgbm as lgb
params = {
          'boosting': 'gbdt',
          'objective': 'regression_l1',
          'metric': {'l1'},
          'num_leaves': 32,
          # 'num_leaves': 16,
          'min_data_in_leaf': 300,
          # 'min_data_in_leaf': 500,
          # 'max_depth': 5,
          'learning_rate': 0.03,
          # "feature_fraction": 0.8,
          # "bagging_fraction": 0.75,
          # "feature_fraction": 0.9,
          # "bagging_fraction": 0.8,
          'feature_fraction': 0.7,
          'bagging_fraction': 0.7,
          'min_data_in_bin':100,
          "lambda_l1": 1e-7,
          'lambda_l2': 1e-7,
          "random_state": 1024,
          "num_threads": 12,
          }

d_train = lgb.Dataset(X_train, y_train)
d_valid = lgb.Dataset(X_valid, y_valid, reference=d_train)

lgb_model = lgb.train(params,
                d_train,
                10000,
                valid_sets=[d_train, d_valid],
                verbose_eval=500,
                early_stopping_rounds=100)
dvalid['lgb_pred'] = lgb_model.predict(dvalid[cols])
calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.lgb_pred.values})

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.068753 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 32031
[LightGBM] [Info] Number of data points in the train set: 342411, number of used features: 129
[LightGBM] [Info] Start training from score 0.455056
Training until validation scores don't improve for 100 rounds
[500]	training's l1: 0.0591138	valid_1's l1: 0.0606629
[1000]	training's l1: 0.05834	valid_1's l1: 0.0601804
[1500]	training's l1: 0.0578747	valid_1's l1: 0.0599995
[2000]	training's l1: 0.057544	valid_1's l1: 0.0599034
[2500]	training's l1: 0.0572907	valid_1's l1: 0.0598324
[3000]	training's l1: 0.057037	valid_1's l1: 0.0597697
[3500]	training's l1: 0.0568802	valid_1's l1: 0.0597449
[4000]	training's l1: 0.0567262	valid_1's l1: 0.0597228
[4500]	training's l1: 0.0565447	valid_1's l1: 0.0596965
[5000]	training's l1: 0.0563927	valid_1's l1: 0.0596735
[5500]	training's l1: 0.0562475	val

0.9162293621948551

In [62]:
gezi.plot.feature_importance(lgb_model, topn=20)

In [63]:
f.feat_name

0                p_cls_pred
1                c_reg_pred
2                    c_pred
3           c_cls_rank_pred
4                    p_pred
5                c_cls_pred
6               p2_cls_pred
7              c_top_pred_1
8               c_rank_pred
9            p_abs_cls_diff
10          c_reg_rank_pred
11            p2_top_pred_1
12             c_top_pred_2
13                  p2_pred
14               p_cls_diff
15             p_top_pred_2
16      c_abs_cls_cls2_diff
17              c_cls2_pred
18         c_cls2_rank_pred
19             c_top_prob_3
20           c_cls_reg_diff
21             c_top_pred_3
22              p_top_sim_5
23          abs_pc_reg_diff
24         p2c_cls_reg_diff
25             p_top_pred_1
26              p2_cls_diff
27          abs_pc_cls_diff
28             p_top_prob_0
29          p2_abs_cls_diff
30            p2_top_pred_2
31         abs_p2c_cls_diff
32       c_abs_cls_reg_diff
33             p_top_pred_6
34          pc_cls_reg_diff
35         abs_p2c_r

In [64]:
FOLDS = 5
cbt_scores, lgb_scores = [], []
scores = []
gezi.try_mkdir('../working/trees')
for fold in tqdm(range(FOLDS)):
  dvalid = df[df.fold==fold]
  dtrain = df[df.fold!=fold]
  X_train = dtrain[cols]
  y_train = dtrain[LABEL_COL]
  X_valid = dvalid[cols]
  y_valid = dvalid[LABEL_COL]
  cbt_model = CatBoostRegressor(**cbt_params)
  cbt_model.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
                verbose=500,
                )  
  cbt_model.save_model(f'../working/trees/{fold}.cbt')
  dvalid['cb_pred'] = cbt_model.predict(dvalid[cols])
  cbt_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cb_pred.values})
  ic(cbt_score)
  
  d_train = lgb.Dataset(X_train, y_train)
  d_valid = lgb.Dataset(X_valid, y_valid, reference=d_train)
  lgb_model = lgb.train(params,
                  d_train,
                  10000,
                  valid_sets=[d_train, d_valid],
                  verbose_eval=500,
                  early_stopping_rounds=100)
  lgb_model.save_model(f'../working/trees/{fold}.lgb')
  dvalid['lgb_pred'] = lgb_model.predict(dvalid[cols])
  
  lgb_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.lgb_pred.values})
  ic(lgb_score)
  score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': (dvalid.cb_pred.values * 0.3 + dvalid.lgb_pred.values * 0.7)})
  ic(score)
  
  cbt_scores.append(cbt_score)
  lgb_scores.append(lgb_score)
  scores.append(score)
  ic(fold, 
     np.asarray(cbt_scores).mean(), 
     np.asarray(lgb_scores).mean(),
     np.asarray(scores).mean(),
     )

  0%|          | 0/5 [00:00<?, ?it/s]

0:	learn: 0.2522445	test: 0.2522445	test1: 0.2522014	best: 0.2522014 (0)	total: 67.4ms	remaining: 11m 14s
500:	learn: 0.0577011	test: 0.0577011	test1: 0.0607874	best: 0.0607874 (500)	total: 34s	remaining: 10m 45s
1000:	learn: 0.0554715	test: 0.0554715	test1: 0.0603776	best: 0.0603776 (1000)	total: 1m 8s	remaining: 10m 13s
1500:	learn: 0.0534726	test: 0.0534726	test1: 0.0603444	best: 0.0603199 (1414)	total: 1m 42s	remaining: 9m 40s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.06031987481
bestIteration = 1414

Shrink model to first 1415 iterations.


[08/05/22 19:47:01] 1620804012.py:20 in <module>
                    cbt_score: 0.9155755035269483


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.057236 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 32031
[LightGBM] [Info] Number of data points in the train set: 342411, number of used features: 129
[LightGBM] [Info] Start training from score 0.455056
Training until validation scores don't improve for 100 rounds
[500]	training's l1: 0.0591138	valid_1's l1: 0.0606629
[1000]	training's l1: 0.05834	valid_1's l1: 0.0601804
[1500]	training's l1: 0.0578747	valid_1's l1: 0.0599995
[2000]	training's l1: 0.057544	valid_1's l1: 0.0599034
[2500]	training's l1: 0.0572907	valid_1's l1: 0.0598324
[3000]	training's l1: 0.057037	valid_1's l1: 0.0597697
[3500]	training's l1: 0.0568802	valid_1's l1: 0.0597449
[4000]	training's l1: 0.0567262	valid_1's l1: 0.0597228
[4500]	training's l1: 0.0565447	valid_1's l1: 0.0596965
[5000]	training's l1: 0.0563927	valid_1's l1: 0.0596735
[5500]	training's l1: 0.0562475	val

[08/05/22 19:48:57] 1620804012.py:34 in <module>
                    lgb_score: 0.9162293621948551
[08/05/22 19:48:59] 1620804012.py:36 in <module>
                    score: 0.9164384885481429
[08/05/22 19:48:59] 1620804012.py:41 in <module>
                    fold: 0
                    np.asarray(cbt_scores).mean(): 0.9155755035269483
                    np.asarray(lgb_scores).mean(): 0.9162293621948551
                    np.asarray(scores).mean(): 0.9164384885481429


...

[08/05/22 20:01:47] 1620804012.py:34 in <module>
                    lgb_score: 0.9210017857764415
[08/05/22 20:01:49] 1620804012.py:36 in <module>
                    score: 0.921358536104917
[08/05/22 20:01:49] 1620804012.py:41 in <module>
                    fold: 3
                    np.asarray(cbt_scores).mean(): 0.9180080927707349
                    np.asarray(lgb_scores).mean(): 0.9185401786145279
                    np.asarray(scores).mean(): 0.9188100897449317


0:	learn: 0.2521270	test: 0.2521270	test1: 0.2522483	best: 0.2522483 (0)	total: 70.5ms	remaining: 11m 45s
500:	learn: 0.0575577	test: 0.0575577	test1: 0.0614648	best: 0.0614648 (500)	total: 34.4s	remaining: 10m 52s
1000:	learn: 0.0552563	test: 0.0552563	test1: 0.0610869	best: 0.0610869 (1000)	total: 1m 8s	remaining: 10m 19s
1500:	learn: 0.0536794	test: 0.0536794	test1: 0.0610580	best: 0.0610522 (1370)	total: 1m 43s	remaining: 9m 45s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.06105220471
bestIteration = 1370

Shrink model to first 1371 iterations.


[08/05/22 20:04:08] 1620804012.py:20 in <module>
                    cbt_score: 0.9091187713116997


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.067719 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 32032
[LightGBM] [Info] Number of data points in the train set: 341935, number of used features: 129
[LightGBM] [Info] Start training from score 0.456522
Training until validation scores don't improve for 100 rounds
[500]	training's l1: 0.0588562	valid_1's l1: 0.0614861
[1000]	training's l1: 0.0581187	valid_1's l1: 0.0610003
[1500]	training's l1: 0.0576907	valid_1's l1: 0.0608042
[2000]	training's l1: 0.0573796	valid_1's l1: 0.060701
[2500]	training's l1: 0.0570748	valid_1's l1: 0.0606048
[3000]	training's l1: 0.0567945	valid_1's l1: 0.0605563
[3500]	training's l1: 0.0566017	valid_1's l1: 0.0605169
[4000]	training's l1: 0.0564326	valid_1's l1: 0.060483
[4500]	training's l1: 0.056255	valid_1's l1: 0.0604473
[5000]	training's l1: 0.0560622	valid_1's l1: 0.0604214
[5500]	training's l1: 0.0559404	va

[08/05/22 20:06:02] 1620804012.py:34 in <module>
                    lgb_score: 0.9098462749002174
[08/05/22 20:06:04] 1620804012.py:36 in <module>
                    score: 0.910049283878711
[08/05/22 20:06:04] 1620804012.py:41 in <module>
                    fold: 4
                    np.asarray(cbt_scores).mean(): 0.9162302284789279
                    np.asarray(lgb_scores).mean(): 0.9168013978716658
                    np.asarray(scores).mean(): 0.9170579285716876


In [65]:
# 0.916967035320258
# fold: 0
#                     np.asarray(cbt_scores).mean(): 0.9162172691455124
#                     np.asarray(lgb_scores).mean(): 0.9167637498752904
#                     np.asarray(scores).mean(): 0.9169839267736672